#Practical 1C#

#Simple network, excitatory and inhibitory reversal potentials#

In previous practicals we implemented Euler's method of numerical integration ourselves. In this practical, we will use make life easier for ourselves by using a Python package called Brian that contains code we can call upon to do numerical integration and many other things needed for simulating spiking neural networks. 

The following code cell installs Brian in the present Colab runtime. Installation is a bit slow, and only needs to be done once per runtime. You can save time by re-running subsequent code cells without reinstalling Brian (but it won't hurt if you do). You can learn more about Brian at https://briansimulator.org/.

**Run the following code cell by pressing the "play" button in the left margin.**

In [ ]:
!pip install brian2

The following code runs a simple network of one excitatory presynaptic neuron and one inhibitory presynaptic neuron, both of which synapse onto a postsynaptic neuron.

We directly specify the spike times of the presynaptic neurons, from which the resulting changes in synaptic conductances and membrane potential in the postsynaptic neuron are calculated.

Some of the important variables in the following code are

**etimes**: spikes times of excitatory presynaptic neuron; initial assignment of spikes at 50 ms and 350 ms 

**itimes**: spikes times of inhibitory presynaptic neuron; initial assignment of spikes at 200 ms and 353 ms

**E_glut**: excitatory reversal potential; initially set at 0 mV

**E_gaba**: inhibitory reversal potential; initially set at -70 mV

**E_l**: resting membrane potential; initially set at -70 mV

**Pe2.i_ext**: current injected into the postsynaptic neuron by an intracellular electrode; initially set at 0 pA

**Run the following code cell by pressing the "play" button in the left margin.**

In [ ]:
from brian2 import *
import matplotlib.pyplot as plt

# Spike times of excitatory presynaptic neuron
etimes = array([50, 350])*ms
eindices = np.zeros(len(etimes))
Pe1 = SpikeGeneratorGroup(3, eindices, etimes)

# Spike times of inhibitory presynaptic neuron
itimes = array([200, 353])*ms
iindices = np.zeros(len(itimes))
Pi1 = SpikeGeneratorGroup(3, iindices, itimes)

# Differential equations for membrane potential, synaptic conductances 
eqs = '''
dv/dt = (-gl*(v-El) - g_glut*(v-E_glut) - g_gaba*(v-E_gaba) + i_ext)/Cm: volt
dg_glut/dt = -g_glut/t_glut : siemens
dg_gaba/dt = -g_gaba/t_gaba : siemens
i_ext : ampere
'''

# Postsynaptic neuron parameters
Cm  = 0.5*nF  		# [nF] membrane capacitance
gl  = 25.0*nS 		# [nS] resting or leak conductance
El  = -70.0*mV		# [mV] resting or leak reversal potential
Vth = -50.0*mV		# [mV] threshold potential
Vr  = -60.0*mV		# [mV] reset potential
tr  = 2.0*ms		  # [ms] refractory time

# Glutamate receptor parameters
E_glut = 0.0*mV   # [mV] excitatory synaptic reversal potential
t_glut = 5.0*ms  	# [ms] excitatory conductance decay time constant

# GABA receptor parameters
E_gaba = -70.0*mV	 # [mV] inhibitory synaptic reversal potential
t_gaba = 5.0*ms		 # [ms] inhibitory conductance decay time constant

# Postsynaptic neuron
Pe2 = NeuronGroup(1, eqs, threshold='v > Vth', reset='v = Vr', refractory=tr, order=2)
Pe2.v = -70.0*mV      # Initial value of postsynaptic membrane potential
Pe2.g_glut = 0.0*nS   # Initial value of excitatory postsynaptic conductance
Pe2.g_gaba = 0.0*nS   # Initial value of inhibitory postsynaptic conductance
Pe2.i_ext = 0.0*pA  # Initial value of injected current 

# Connect excitatory presynptic neuron to postsynaptic neuron
Cpe = Synapses(Pe1, Pe2, on_pre='g_glut += we12') 
Cpe.connect(condition='i==j')
we12 = 25.0*nS      # Postsynaptic excitatory conductance increase due to a presynaptic spike 

# Connect inhibitory presynaptic neuron to postsynaptic neuron
Cpi = Synapses(Pi1, Pe2, on_pre='g_gaba += wi12') 
Cpi.connect(condition='i==j')
wi12 = 50.0*nS     # Postsynaptic inhibitory conductance increase due to a presynaptic spike

# Data to record when simulation runs
mon_Pe1_sp = SpikeMonitor(Pe1)
mon_Pi1_sp = SpikeMonitor(Pi1)
mon_Pe2_sp = SpikeMonitor(Pe2)
mon_Pe2_vm = StateMonitor(Pe2, 'v', record=0)
mon_Pe2_ge = StateMonitor(Pe2, 'g_glut', record=0)
mon_Pe2_gi = StateMonitor(Pe2, 'g_gaba', record=0)

# Run simulation
duration = 500.0*ms
run(duration)

# Plot traces
fig1 = plt.figure(figsize=(10,10))
# Plot presynaptic spike times
ax1 = fig1.add_subplot(3,1,1)
ax1.plot(mon_Pe1_sp.t/ms,  0.5+mon_Pe1_sp.i, '.', markersize = 15, color = 'blue', label = 'Excitatory spikes')
ax1.plot(mon_Pi1_sp.t/ms, -0.5+mon_Pi1_sp.i, '.', markersize = 15, color = 'red', label = 'Inhibitory spikes')
xlim(0, duration/ms)
ylim(-1,1)
plt.xticks([]), plt.yticks([])
plt.title('Presynaptic spike times')
ax1.legend()
# Plot synaptic conductances
ax2 = fig1.add_subplot(3,1,2)
plot(mon_Pe2_ge.t/ms, mon_Pe2_ge.g_glut[0]/nS, lw = 3, color = 'blue', label = 'Excitatory conductance')
plot(mon_Pe2_gi.t/ms, mon_Pe2_gi.g_gaba[0]/nS, lw = 3, color = 'red', label = 'Inhibitory conductance')
plt.xticks([])
ax2.set_ylabel('G (nS)')
plt.title('Synaptic conductances')
ax2.legend()
# Plot postsynaptic membrane potential
ax3 = fig1.add_subplot(3,1,3)
# Draw spikes in traces
Pe2_vm = mon_Pe2_vm[0].v[:]
for t in mon_Pe2_sp.t:
    i = int(t / defaultclock.dt)
    Pe2_vm[i] = 0*mV
plot(mon_Pe2_vm.t/ms, Pe2_vm/mV, lw = 3, color = 'black')
ax3.set_ylabel('Vm (mV)')
ax3.set_xlabel('Time (ms)')
plt.title('Postsynaptic membrane potential')
plt.show()

**Exercises**

**1. Recall that the reversal potential is a property of an ion channel (and extracellular and intracellular ion concentrations). When a channel is open, current will flow through it to drive the membrane potential towards the channel's reversal potential.**

With the initial settings, the simulation consists of the excitatory neuron firing at 50 ms, the inhibitory neuron firing at 200 ms, then the excitatory neuron and inhibitory neuron firing nearly simultaneously at around 350 ms. 

The inhibitory neuron spike at 200 ms causes postsynaptic GABA receptors to open, which is plotted as an increase in inhibitory synaptic cnductance, but there is no change in membrane potential, because the membrane potential is already at the GABA reversal potential. The inhibition causes the depolarization after the excitatory neuron spike to be less when it is nearly coincident with the inhibitory neuron spike (around 350 ms), than when it occurs alone (at 50 ms).

Variation in intracellular or extracellular chloride conentrations lead to variations in the GABA reversal potential. **Modify the code above so that the GABA reversal potential (E_gaba) is either slightly more negative (eg. -75 mV) or slightly more positive (eg. -65 mV), and show that a hyperpolarization or depolarization can follow the inhibitory neuron spike.** 

Regardless of whether the GABA reversal potential is slightly more positive or negative, as long as the GABA reversal potential is more negative than the depolarization caused when the excitatory neuron spike occurs alone, the near coincidence of excitatory and inhibitory neuron spikes reduces the depolarization after the excitatory neuron spike. 

**2. Reset E_gaba to -70 mV. Inject current into the neuron by changing Pe2.i_ext from 0 pA to 100 pA.** This will depolarize the membrane potential, so that the GABA reversal potential is more negative than the membrane potential. Thus the inhibitory neuron spike at 200 ms will hyperpolarize the membrane potential.

**3. Change the spike times of the excitatory and inhibitory presynaptic neurons (contained in etimes and itimes).**  You can add or remove spikes.  Note that if you specify spikes to occur at more than 500 ms, you also have to increase the **duration** of the simulation, which is initially set at 500 ms.